In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
url = 'https://500.farm/vastai-exporter/machines'

r = requests.get(url)
r.raise_for_status()

offers  = pd.DataFrame(r.json()["offers"])
offers['timestamp'] = pd.to_datetime(r.json()['timestamp'])
print(offers.timestamp[0])
offers